In [25]:
import numpy as np
import qutip as qt
import sympy as sp



## Numerical Calculation Linbladian

In [39]:
p,q,r,s = sp.symbols('\\rho_{11} \\rho_{12} \\rho_{21} \\rho_{22}')
d,o,k = sp.symbols('\Delta \Omega \gamma')
w,x,y,z = sp.symbols('c_{11} c_{12} c_{21} c_{22}')

j = sp.Symbol('j')

In [42]:
rho = sp.Matrix([[p,q],[r,s]])
ham = sp.Matrix([[-d,o],[o,d]])
c = sp.Matrix([[w,x],[y,z]])
cdag = c.H

Matrix([
[conjugate(c_{11}), conjugate(c_{21})],
[conjugate(c_{12}), conjugate(c_{22})]])

In [45]:
coherent_term = ham*rho-rho*ham
coherent_term.subs({p:1,q:0,r:0,s:0,o:1})

Matrix([
[0, -1],
[1,  0]])

In [55]:
dis_1 = 2*c*rho*cdag
dis_2 = rho*cdag*c
dis_3 = cdag*c*rho
dissipator = 0.5*(dis_1-dis_2-dis_3)

dissipator.subs({p:1,q:0,r:0,s:0,w:0,x:0,y:1,z:0})

Matrix([
[-1.0,   0],
[   0, 1.0]])

In [56]:
# Linbladian?
j*coherent_term.subs({p:1,q:0,r:0,s:0,o:1})+k.subs({k:1})*dissipator.subs({p:1,q:0,r:0,s:0,w:0,x:0,y:1,z:0})

Matrix([
[-1.0,  -j],
[   j, 1.0]])

# Liouvillian fron Qutip

In [73]:
def hamiltonian_ising(N,omega,delta,V):
  sx = qt.operators.sigmax()
  sy = qt.operators.sigmay()
  sz = qt.operators.sigmaz()
  I = qt.operators.identity(2)

  ising_ham = qt.tensor([I*0]*N)
  for i in range(N):
    hz = qt.tensor([sz if j==i else I for j in range(N)])
    hx = qt.tensor([sx if j==i else I for j in range(N)])
    ising_ham += -1*delta*hz+omega*hx
    for j in range(i + 1, N):
        # Interaction term between qubit i and qubit j
        #print(i,j)
        hzz = qt.tensor([sz if k == i or k == j else I for k in range(N)]) # assume obc
        ising_ham += (V/N)*hzz
  
  #print(hz,hx,hzz)
  
  return(ising_ham)

In [75]:
ham = hamiltonian_ising(1,1,1,3)
ham

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[-1.  1.]
 [ 1.  1.]]

In [77]:
psi0 = qt.tensor(qt.basis(2,0),qt.basis(2,1))
sm = qt.operators.sigmam()
I = qt.operators.identity(2)
kappa = 1
N=1
c_ops = [qt.tensor([sm if i==j  else I for j in range(N)]) for i in range(N)]
c_ops = [np.sqrt(kappa) * c_op for c_op in c_ops]
L = qt.liouvillian(ham, c_ops).full()

In [81]:
c_ops

[Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
 Qobj data =
 [[0. 0.]
  [1. 0.]]]

In [78]:
L

array([[-1. +0.j,  0. -1.j,  0. +1.j,  0. +0.j],
       [ 0. -1.j, -0.5-2.j,  0. +0.j,  0. +1.j],
       [ 0. +1.j,  0. +0.j, -0.5+2.j,  0. -1.j],
       [ 1. +0.j,  0. +1.j,  0. -1.j,  0. +0.j]])

In [80]:
psi0*psi0.dag()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]